In [11]:
import sys
import ee
%run Sentinel2b.ipynb
ee.Authenticate()
ee.Initialize()

modulename = 'ee'
if modulename not in sys.modules:
    # import GEE and Authenticate, token or log in will be asked from web browser
    import ee
    # TO DO : UNCOMMENT THIS IF IT IS THE FIRST TIME AUTHINTICATE GEE ON THIS MACHINE
    # ee.Authenticate()
    ee.Initialize()
# else:
    # google earth engine already imported and authenticated


modulename = 'ipynb_MapVisualisation'
if modulename not in sys.modules:
    %run MapVisualisation.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_MapVisualisation'] = None
# else
    # Utils modules has already been loaded somewhere else


modulename = 'ipynb_Sentinel2b'
if modulename not in sys.modules:
    %run Sentinel2b.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_Sentinel2b'] = None
# else
    # Utils modules has already been loaded somewhere else


Sentinel2b class imported


In [12]:

# Area of Interest
AOI = ee.Geometry.Polygon(
        [[[-2.97404756802236, 41.99472813320794],
          [-2.97404756802236, 41.8107480842418],
          [-2.66643038052236, 41.8107480842418],
          [-2.66643038052236, 41.99472813320794]]])

# cloudmask01 = getCloudMask(median)

START_DATE = '2019-01-01'
END_DATE = '2019-12-31'



masks = { 'gsw': 1}#,'forestMask': {'buffer':10, 'startDate':'2000-01-01', 'endDate':'2022-12-31',}} # # , 'lmask': 30,

printAvailableIndicesS2()

sentinel2Info = {
    "identifier": 'sentinel-2',                                              
    "collection": "COPERNICUS/S2_SR_HARMONIZED",
    #"launchDate" : "2015-05-23",
    #"retirmentDate":todayDate, #not retired yet
    "startDate"   : '2019-01-01',
    "endDate"     : '2019-12-31',
    #"added"     : False,
    "clouds"    : 50,
    "selectedBand": ['B8', 'B4'],
    "selectedIndices": ['ndvi']
} 


s2 = Sentinel2(AOI, sentinel2Info, masks)


# s2.byMonth(2020)

median = s2.getMedian()
bandNames = median.bandNames()
print('Band names medianS2: ', bandNames.getInfo())

# Code used for testing cloud masks - Cloud masking used by default so no need to worry about this code
#med = s2.getCollection().first()
#clouds = med.select('clouds').selfMask()
#shadows = med.select('shadows').selfMask()
#dark_pixels = med.select('dark_pixels')  # .selfMask()
#probability = med.select('probability')
#cloudmask = med.select('cloudmask')
#cloudmaskMasked = cloudmask  # .selfMask()
#cloud_transform = med.select('cloud_transform')

# Selecting RGB bands for visualisation purposes
#medianS2 = median.select(['B4', 'B3', 'B2'])


# Calculate NDVI on the median image
medianB84 = median.select(['B4', 'B8'])
medianB84NDVI = s2.addNDVIimg(medianB84)


'''
# export image  to GEE - adding maxPixels is a bad habit, used carefully
task = ee.batch.Export.image.toDrive(**{
    'image': medianB84NDVI,
    'description': 'medRGB1_GB1_2019',
    'folder': 'earth_engine_demos',
    'scale': 10,
    'region': AOI.getInfo()['coordinates'],
    'maxPixels': 1549491660
})
task.start()
'''


    Key                                   Index                Formula
0  ndvi  Normalised Difference Vegetation Index  (B8 - B4) / (B8 + B4)
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE', 'probability', 'clouds', 'dark_pixels', 'cloud_transform', 'shadows', 'cloudmask']  allbands
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B11
B12
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'] ******
selectedBandsNIndices: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
Band names medianS2:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']


"\n# export image  to GEE - adding maxPixels is a bad habit, used carefully\ntask = ee.batch.Export.image.toDrive(**{\n    'image': medianB84NDVI,\n    'description': 'medRGB1_GB1_2019',\n    'folder': 'earth_engine_demos',\n    'scale': 10,\n    'region': AOI.getInfo()['coordinates'],\n    'maxPixels': 1549491660\n})\ntask.start()\n"

In [13]:
%run MapVisualisation.ipynb
# These are the coordinates of Spain
my_map = folium.Map(location=[42, -3], zoom_start=11)
# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)


def update_cld_shdw_mask(img):
    print('test')



# Add Land Mask to the map
my_map.add_ee_layer(medianB84NDVI, {
                    'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'median')
my_map.add_ee_layer(AOI, {}, 'AOI')



# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)


MapVisualisation class imported - uses folium library
Could not display median
